<a href="https://colab.research.google.com/github/LXHayato/Sawing_Reinforcement_L/blob/main/sawing_Reinforcement_L.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#とりあえずカテドラルローブのみ
#Q学習

class QL_Agent
  def_init_(self,alpha=0.2, epsilon=0.1, gamma=0.99, actions=None, observation=None):
    self.alpha=alpha
    self.gamma=gamma
    self.epsilon=epsilon
    self.rewrd_history=[]
    self.actions=actions
    self.state=str(observation)
    self.ini_state=str(observation)
    self.previous_state=None
    self.previous_action=None
    self.q_values=self._init_q_values()

    